In [16]:
# !wget https://huggingface.co/datasets/mesolitica/pseudolabel-science-large-v3-timestamp/resolve/main/dataset-science.jsonl

In [8]:
import json
from tqdm import tqdm

data = []
with open('dataset-science.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        l.pop('predict_en', None)
        data.append(l)

In [10]:
len(data)

885624

In [21]:
import re

pattern_pair = r'<\|(\d+\.\d+)\|>(.*?)<\|(\d+\.\d+)\|>'
matches = re.findall(pattern_pair, '<|0.00|> kerajaan persekutuan<|1.46|><|1.46|> dan banyak masalah hubungan<|3.96|><|3.96|> antara kerajaan negeri dan')
matches

[('0.00', ' kerajaan persekutuan', '1.46'),
 ('1.46', ' dan banyak masalah hubungan', '3.96')]

In [22]:
import string

punct = set(string.punctuation)

def remove_punct(s):
    return ''.join([c for c in s if c not in punct])

def remove_duplicate(string, n = 3):
    splitted = string.split()
    n = [splitted[i: i + n] for i in range(0, len(splitted), n)]
    already = set()
    dedup = []
    for n_ in n:
        original_n = ' '.join(n_)
        n_ = ' '.join(n_).lower()
        n_ = remove_punct(n_)
        if n_ not in already:
            dedup.append(original_n)
            already.add(n_)
    return ' '.join(dedup)

In [4]:
import math

def round_to_nearest_0_02(number):
    return round(number * 50) / 50

In [7]:
selected = [
    'terima kasih kerana menonton',
    'terima kasih',
    'thank you for watching',
    'Thank you.'
]

In [11]:
data[0]

{'score_en': 0.87890625,
 'filename': 'chunk/mp3-16k-0-0_000.mp3',
 'output_en': '<|startoftranscript|><|en|><|transcribe|><|0.00|> Thank you.<|29.96|><|endoftext|>'}

In [26]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from transformers import AutoTokenizer, WhisperConfig
from sklearn.feature_extraction.text import CountVectorizer
import mp
import copy

minimum_score = 5

def loop(files):
    
    data, _ = files
    results = []

    for i in tqdm(range(len(data))):
            
        if data[i]['score_en'] > minimum_score:
            t = data[i]['output_en'].strip()
            if t.split('|>')[-1] != '':
                t += rounded_num

            matches = re.findall(pattern_pair, t)
            rs = []
            for match in matches:
                l = float(match[0])
                r = float(match[2])
                t_ = match[1]
                rt_ = re.sub('[^a-z ]+', '', t_.lower()).strip()
                if (r - l > 3) and any([s == rt_ for s in selected]):
                    # print(audio_filename, t_)
                    t_ = ''
                else:
                    try:
                        dense = CountVectorizer(ngram_range = (3,3)).fit_transform([t_]).todense()
                        repeat = (dense > 3).sum() >= 1
                        if repeat:
                            t_ = remove_duplicate(t_)
                    except:
                        if len(t_) > 100:
                            t_ = remove_duplicate(t_)
                rs.append(f'<|{match[0]}|>{t_}<|{match[2]}|>')
            rs = ''.join(rs)
            t = f'<|startoftranscript|><|en|><|transcribe|>{rs}<|endoftext|>'
            d = {
                'new_text': t,
                'audio_filename': data[i]['filename'],
            }
            results.append(d)
    return results

In [27]:
loop((data[:10], 0))

100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 430.47it/s]


[{'new_text': "<|startoftranscript|><|en|><|transcribe|><|0.00|> insightful reporting and engaging storytelling with deep roots in television and radio.<|5.96|><|6.48|> She's the former host of the national news program Here and Now, and she continues to host the<|11.82|><|11.82|> podcast Truth Be Told, which recently launched a new season called She Has a Name. Tanya,<|19.34|><|19.46|> welcome to the Minor Consult. It's great to have you here today. It's so great to be here,<|24.54|><|24.54|> Dean. I'm so excited. So, Tanya, you're in the news business, so let's start with your latest<|29.96|><|endoftext|>",
  'audio_filename': 'chunk/mp3-16k-0-0_001.mp3'},
 {'new_text': '<|startoftranscript|><|en|><|transcribe|><|0.00|> News. You recently launched a podcast called She Has a Name, which has an intensely personal<|6.68|><|6.68|> connection. Can you tell us about the origins of the podcast and your journey producing<|11.30|><|11.30|> it with your nephew?<|12.22|><|14.10|> Yes, as you sa

In [30]:
r = mp.multiprocessing(data, loop, cores = 10)

100%|██████████████████████████████████████████████████████████████████████████| 88562/88562 [03:15<00:00, 453.27it/s]


In [31]:
len(r)

850638

In [32]:
r[0]

{'new_text': "<|startoftranscript|><|en|><|transcribe|><|0.00|> insightful reporting and engaging storytelling with deep roots in television and radio.<|5.96|><|6.48|> She's the former host of the national news program Here and Now, and she continues to host the<|11.82|><|11.82|> podcast Truth Be Told, which recently launched a new season called She Has a Name. Tanya,<|19.34|><|19.46|> welcome to the Minor Consult. It's great to have you here today. It's so great to be here,<|24.54|><|24.54|> Dean. I'm so excited. So, Tanya, you're in the news business, so let's start with your latest<|29.96|><|endoftext|>",
 'audio_filename': 'chunk/mp3-16k-0-0_001.mp3'}

In [33]:
with open('prepared-en.jsonl', 'w') as fopen:
    for r_ in r:
        fopen.write(f'{json.dumps(r_)}\n')

In [34]:
!tail -n 3 prepared-en.jsonl

{"new_text": "<|startoftranscript|><|en|><|transcribe|><|0.00|> And here we can specify a bunch of other properties.<|3.00|><|3.56|> So we can specify the mass of our player.<|6.30|><|6.70|> I'm going to use 100 as the value.<|8.66|><|9.04|> How I came up with this was just by tweaking.<|11.28|><|11.28|> And we can also set the jump force.<|13.90|><|14.32|> So this is when we're going to implement jumping.<|16.00|><|16.70|> You can set up in Kaboom.js the jump force.<|19.08|><|19.28|> And here 320 was a value which I thought was powerful enough<|23.66|><|23.66|> so that the player can have a nice jump,<|26.76|><|27.12|> but not too much so that they hit the ceiling<|29.96|><|endoftext|>", "audio_filename": "chunk/mp3-16k-11-154_203.mp3"}
{"new_text": "<|startoftranscript|><|en|><|transcribe|><|0.00|> of the levels so here we go and uh also we are also going to add the double jump component so<|9.36|><|9.36|> double jump is a component that is provided by kaboom that allows you to easil

In [35]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj='prepared-en.jsonl',
    path_in_repo='prepared-en.jsonl',
    repo_id="mesolitica/pseudolabel-science-large-v3-timestamp",
    repo_type="dataset",
)

prepared-en.jsonl:   0%|          | 0.00/568M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/pseudolabel-science-large-v3-timestamp/commit/83f2170ffade0e20a11115ca07e618027565ed8d', commit_message='Upload prepared-en.jsonl with huggingface_hub', commit_description='', oid='83f2170ffade0e20a11115ca07e618027565ed8d', pr_url=None, pr_revision=None, pr_num=None)